Reference

- https://github.com/adapter-hub/adapters
- https://docs.adapterhub.ml/loading.html
- https://docs.adapterhub.ml/adapter_composition.html


-https://huggingface.co/docs/transformers/peft

In [1]:
!sudo -H pip install -Uq transformers

In [1]:
#!pip install peft==0.3.0

In [2]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import pandas as pd
import torch

## 1. With base model in quantized form

In [3]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


model_name = "facebook/opt-iml-max-1.3b"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
#     load_in_8bit=False,
#     torch_dtype=torch.float16
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
from peft import PeftModel, PeftConfig

In [6]:
adapter1 = "/data/quantization-trials/checkpoint-50"
adapter2 = "/data/quantization-trials/checkpoint-10"

In [10]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules = ["q_proj", "v_proj"],
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)

In [11]:
peft_model = get_peft_model(model, peft_config)

In [12]:
peft_model.load_adapter(adapter1, adapter_name = 'adapter1')

_IncompatibleKeys(missing_keys=['base_model.model.model.decoder.embed_tokens.weight', 'base_model.model.model.decoder.embed_positions.weight', 'base_model.model.model.decoder.final_layer_norm.weight', 'base_model.model.model.decoder.final_layer_norm.bias', 'base_model.model.model.decoder.layers.0.self_attn.k_proj.weight', 'base_model.model.model.decoder.layers.0.self_attn.k_proj.bias', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.weight', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.bias', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.decoder.layers.0.self_attn.q_proj.weight', 'base_model.model.model.decoder.layers.0.self_attn.q_proj.bias', 'base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.m

In [14]:
# using load_adapter

peft_model.load_adapter(adapter2, adapter_name = 'adapter2')

_IncompatibleKeys(missing_keys=['base_model.model.model.decoder.embed_tokens.weight', 'base_model.model.model.decoder.embed_positions.weight', 'base_model.model.model.decoder.final_layer_norm.weight', 'base_model.model.model.decoder.final_layer_norm.bias', 'base_model.model.model.decoder.layers.0.self_attn.k_proj.weight', 'base_model.model.model.decoder.layers.0.self_attn.k_proj.bias', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.weight', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.bias', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.adapter1.weight', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.adapter1.weight', 'base_model.model.model.decoder.layers.0.self_attn.q_proj.weight', 'base_model.model.model.decoder.layers.0.self_attn.q_proj.bias', 'base_model.model

In [15]:
%%time

peft_model.set_adapter('adapter1')

CPU times: user 0 ns, sys: 2.87 ms, total: 2.87 ms
Wall time: 2.88 ms


In [18]:
# %%time
# model_to_merge = PeftModel.from_pretrained(model,adapter1)

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 376 ms


In [11]:
# %%time
# model_to_merge2 = PeftModel.from_pretrained(model,adapter2)
# model_to_merge2.to('cuda')

In [16]:
def predict_template_query_v1(user_query):
    inp = query_template_v1.format(context=context,
                                   user_query=user_query,
                                  date_input=date_input)
    _inputs = tokenizer.encode(inp, return_tensors="pt").to('cuda')
    outputs = peft_model.generate(input_ids=_inputs, max_length= 900, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    return output_new.split('\n[/MQL]')[0], output

In [17]:
import time

In [26]:
start = time.time()
user_query = 'what is purchase across segments'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print("time taken : ", time.time()-start)

user query:  what is purchase across segments
----------------------------------------------------------------------------------------------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 78.00 MiB (GPU 0; 15.60 GiB total capacity; 14.58 GiB already allocated; 20.94 MiB free; 14.90 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [23]:
output

"{'"

In [18]:
context = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

date_input = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [19]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [ ]:
# model = AutoAdapterModel.from_pretrained("roberta-base")


model.load_adapter("AdapterHub/roberta-base-pf-imdb", source="hf", set_active=True)

print(model(**tokenizer("This works great!", return_tensors="pt")).logits)